<a href="https://colab.research.google.com/github/gitHubAndyLee2020/LLM_Reward_Model_Research/blob/main/Mistral_7B_Instruct_v0_1_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model Initialization

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a classic mayonnaise recipe that I've found to work quite well:

Ingredients:

* 2 large egg yolks
* 1 teaspoon Dijon mustard
* 2 tablespoons white wine vinegar
* 1/2 teaspoon salt
* 1/4 cup olive oil
* 1/4 to 1/2 cup additional oil, as needed

Instructions:

1. In a mixing bowl, whisk together the egg yolks, mustard, vinegar, and salt until well combined.
2. Slowly drizzle in the olive oil, whisking constantly to keep the mixture emulsified.
3. If the mayonnaise seems thick or too curdled, add a bit of water or additional oil to thin it out.
4. Taste and adjust the seasoning, as needed.
5. Serve and enjoy!

Of course, there are many variations of mayonnaise out there, and you can add your own touch by including thin

In [4]:
%%time

sys_prompt = "You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old."
prompt = "What kind of task would test someone's ability to perform physical reasoning?"

prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + prompt + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format

messages = [
    {"role": "user", "content": input_text},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] <|im_start|>system
You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.<|im_end|>
<|im_start|>user
What kind of task would test someone's ability to perform physical reasoning?<|im_end|>
<|im_start|>assistant
 [/INST] Physical reasoning involves using your body and your understanding of how things move and work to solve problems. Some tasks that might test your ability to perform physical reasoning are:

1. Building something with blocks or LEGOs. This helps you understand how to make structures that are strong and stable.
2. Playing a game like Simon Says or Red Light, Green Light. This helps you understand the relationship between space and time, and how to move your body in different ways to avoid being caught.
3. Doing push-ups or hanging from a pull-up bar. These tasks help you understand how to use your strength and control your body to move in different positions.
4. Riding a bike or playing a game like tag on th

### Dataset Initialization

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

# The dataset identifier is usually in the format "user/dataset_name"
dataset_identifier = "Open-Orca/OpenOrca"
dataset = load_dataset(dataset_identifier)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
import pandas as pd

# Convert the "train" split to a DataFrame
df_train = dataset["train"].to_pandas()

# Display the first few rows
df_train.head()

,id,system_prompt,question,response
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...
3,t0.408370,You are an AI assistant. You will be given a t...,Please answer the following question: I want t...,"Based on the passage, discuss the primary moti..."
4,cot.86217,You are an AI assistant that helps people find...,James runs a TV show and there are 5 main char...,"James pays the minor characters $15,000 each e..."


### Generating Responses

In [55]:
def generate_response(sys_prompt, prompt):
  prefix = "<|im_start|>"
  suffix = "<|im_end|>\n"
  sys_format = prefix + "system\n" + sys_prompt + suffix
  user_format = prefix + "user\n" + prompt + suffix
  assistant_format = prefix + "assistant\n"
  input_text = sys_format + user_format + assistant_format

  messages = [
      {"role": "user", "content": input_text},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [56]:
import re

def extract_response(text):
  result = re.search(r'\[/INST\](.*?)</s>', text, re.DOTALL)
  if result:
      extracted_text = result.group(1).strip()
      return extracted_text
  else:
      print("Pattern not found.")
      return ''

In [57]:
def get_response(sys_prompt, prompt):
  response = generate_response(sys_prompt, prompt)
  return extract_response(response)

In [58]:
import os

def generate_answers(df_train, START_IDX, END_IDX):
    # Check if data folder exists. If not, create it.
    if not os.path.exists("data"):
        os.makedirs("data")

    for idx in range(START_IDX, END_IDX):
        sys_prompt = df_train["system_prompt"][idx]
        prompt = df_train["question"][idx]
        answer = get_response(sys_prompt, prompt)
        with open(f"data/{idx}_mistral_7b_instruct_v0_1.txt", "w", encoding="utf-8") as f:
            f.write(answer)

In [59]:
START_IDX = 0
END_IDX = 100

In [60]:
%%time

generate_answers(df_train, START_IDX, END_IDX)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

Pattern not found.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

CPU times: user 9min 3s, sys: 4.51 s, total: 9min 8s
Wall time: 9min 7s


In [61]:
!zip -r data.zip data/

  adding: data/ (stored 0%)
  adding: data/56_mistral_7b_instruct_v0_1.txt (deflated 32%)
  adding: data/30_mistral_7b_instruct_v0_1.txt (deflated 50%)
  adding: data/28_mistral_7b_instruct_v0_1.txt (deflated 54%)
  adding: data/27_mistral_7b_instruct_v0_1.txt (deflated 26%)
  adding: data/17_mistral_7b_instruct_v0_1.txt (deflated 56%)
  adding: data/54_mistral_7b_instruct_v0_1.txt (deflated 30%)
  adding: data/73_mistral_7b_instruct_v0_1.txt (deflated 57%)
  adding: data/21_mistral_7b_instruct_v0_1.txt (deflated 46%)
  adding: data/19_mistral_7b_instruct_v0_1.txt (deflated 4%)
  adding: data/61_mistral_7b_instruct_v0_1.txt (deflated 28%)
  adding: data/7_mistral_7b_instruct_v0_1.txt (deflated 59%)
  adding: data/86_mistral_7b_instruct_v0_1.txt (deflated 45%)
  adding: data/53_mistral_7b_instruct_v0_1.txt (deflated 62%)
  adding: data/62_mistral_7b_instruct_v0_1.txt (deflated 62%)
  adding: data/74_mistral_7b_instruct_v0_1.txt (deflated 51%)
  adding: data/18_mistral_7b_instruct_v0_1.t

### Debug

In [ ]:
# If you get "NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968" error, run this to fix the problem

import locale
locale.getpreferredencoding = lambda: "UTF-8"